# Compute Nino3.4 DJF index for each model, and save to file

In [ ]:
# Computes Indexes without caring about branch years

In [267]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.signal import detrend
from matplotlib import pyplot as plt
from eofs.xarray import Eof
from scipy import signal
import pandas as pd
import xarray as xr
import xesmf as xe
import intake
import pprint 
import util 

# choose where to load data from:
load_data_from = 'cloud'
#load_data_from = 'glade'

if load_data_from == 'glade':
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
    file = 'available_data.txt'
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
    file = 'available_data_cloud.txt'

In [268]:
# pick only models with at least 496 yrs in piControl
minyrs_control = 496;
# models with fewer years often missed future scenarios, so they are not so interesting for us

# load table:
data_table = pd.read_table(file,index_col=0)
models_used = data_table['piControl (yrs)'][data_table['piControl (yrs)'] >= minyrs_control].index
print(models_used)

Index(['BCC-CSM2-MR', 'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'E3SM-1-0',
       'EC-Earth3', 'EC-Earth3-Veg', 'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL',
       'HadGEM3-GC31-MM', 'UKESM1-0-LL', 'MRI-ESM2-0', 'GISS-E2-1-G', 'CESM2',
       'CESM2-WACCM', 'GFDL-ESM4', 'SAM0-UNICON', 'MCM-UA-1-0'],
      dtype='object')


In [ ]:
##### Possible missing data in cloud: #####
# 'E3SM-1-0': historical ensemble members 1 and 4 seems to have some missing years in the end
# 'EC-Earth3': historical r24i1p1f1 has some missing years in the end
# 'MRI-ESM2-0': ensemble members 2-5 for ssp245 has only 16 years (no more years are available at ESGF)
# 'GISS-E2-1-G': 4 of the piControl members contain no data. Same problem on Glade, so probably no data at ESGF either.. 
# 'CESM2-WACCM': 'ssp370': members 2 and 3 end in year 2055 (also at ESGF)
# 'GFDL-ESM4': missing historical in cloud, but this seems to be available on Glade. Glade piControl is missing on of the 100yr files
##### For this model, we would have to load piControl from cloud, then historical from Glade, and ssp's from cloud


## Choose what model to use 

In [385]:
model = models_used[18]
#model = models_used[13]
model

'MCM-UA-1-0'

In [331]:
data_table.loc[model]

piControl (ens.mem.)         1
historical (ens.mem.)        2
ssp126 (ens.mem.)          NaN
ssp245 (ens.mem.)            1
ssp370 (ens.mem.)            1
ssp585 (ens.mem.)            1
abrupt-4xCO2 (ens.mem.)      1
piControl (yrs)            500
historical (yrs)           165
ssp126 (yrs)               NaN
ssp245 (yrs)                86
ssp370 (yrs)                86
ssp585 (yrs)                86
abrupt-4xCO2 (yrs)         500
Name: MCM-UA-1-0, dtype: object

In [332]:
# what experiments does this model have that we want to study?
if any(data_table.loc[model][:6] == 'data problem') == False:
    exp_list = [exp[:-11] for exp in data_table.loc[model][:6].index if float(data_table.loc[model][:6][exp]) > 0]
else:
    exp_list = []
    for exp in (data_table.loc[model][:6].index):
        if  (data_table.loc[model][:6][exp] != 'data problem'):
            exp_list = np.append(exp_list, exp[:-11])
print(exp_list)   

['piControl', 'historical', 'ssp245', 'ssp370', 'ssp585']


In [333]:
exp_keys = {}; datasets = {}

for exp in exp_list:
#for exp in [exp_list[1]]:
    print(exp)
    #cat = col.search(experiment_id = exp, source_id = model, variable_id='ts', table_id='Amon', member_id = 'r1i1p1f1')
    cat = col.search(experiment_id = exp, source_id = model, variable_id='ts', table_id='Amon') 
        
    dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
    for key in dset_dict.keys():
        exp_keys[exp] = key
        datasets[key] = dset_dict[key]

exp_keys

piControl
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
historical
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp245
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp370
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp585
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)


{'piControl': 'CMIP.UA.MCM-UA-1-0.piControl.Amon.gn',
 'historical': 'CMIP.UA.MCM-UA-1-0.historical.Amon.gn',
 'ssp245': 'ScenarioMIP.UA.MCM-UA-1-0.ssp245.Amon.gn',
 'ssp370': 'ScenarioMIP.UA.MCM-UA-1-0.ssp370.Amon.gn',
 'ssp585': 'ScenarioMIP.UA.MCM-UA-1-0.ssp585.Amon.gn'}

In [334]:
# load a dataset for manual calendar check:
# if other than noleap, above function must be changed
exp = exp_list[0]; print(exp)
key = exp_keys[exp]
exp_datasets = datasets[key]
members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)

ds = exp_datasets.sel(member_id = members_sorted[0])
#print(ds.time)

#if load_data_from == 'glade'
# Time formats for piControl:
# 'BCC-CSM2-MR': cftime.DatetimeNoLeap
# 'FGOALS-g3': cftime.DatetimeNoLeap # missing files
# 'CanESM5': cftime.DatetimeNoLeap # missing ensemble members
# 'CNRM-CM6-1': cftime.DatetimeGregorian
# 'CNRM-ESM2-1': cftime.DatetimeGregorian # ssp126 have too many time points.
# 'E3SM-1-0': cftime.DatetimeNoLeap # missing files
# 'EC-Earth3': Timestamp('2259-01-16 12:00:00'), ... , cftime.DatetimeProlepticGregorian(2759,..
# 'EC-Earth3-Veg': Timestamp('1852-01-16 12:00:00'), ... , cftime.DatetimeProlepticGregorian(2347,..
# 'IPSL-CM6A-LR': Timestamp('1850-01-16 12:00:00'), ... , cftime.DatetimeGregorian(3049,..
# 'MIROC-ES2L': Timestamp('1850-01-16 12:00:00'), ... , cftime.DatetimeGregorian(2349,
# 'MIROC6': cftime.DatetimeGregorian # code below works, but is not accounting for leap years yet
# 'UKESM1-0-LL': cftime.Datetime360Day # code below works, but must be adjusted for 360day calendar
# 'MRI-ESM2-0': cftime.DatetimeProlepticGregorian # More data should be requested
# 'GISS-E2-1-G': cftime.DatetimeNoLeap # ok, but has no ssp (not in ESGF either)
# 'GISS-E2-1-H': cftime.DatetimeNoLeap # ok, but has no ssp (not in ESGF either)
# 'CESM2': cftime.DatetimeNoLeap 
# 'CESM2-WACCM': cftime.DatetimeNoLeap
# 'GFDL-CM4': cftime.DatetimeNoLeap
# 'SAM0-UNICON': cftime.DatetimeNoLeap
if model in ['BCC-CSM2-MR', 'FGOALS-g3', 'CanESM5', 'E3SM-1-0', 'GISS-E2-1-G', 'GISS-E2-1-H', 'CESM2', 'CESM2-WACCM', 'GFDL-CM4', 'SAM0-UNICON', 'GFDL-ESM4', 'MCM-UA-1-0']:
    ds_calendar = 'noleap'
elif model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6']:
    ds_calendar = 'gregorian'
elif model in ['EC-Earth3', 'EC-Earth3-Veg', 'MRI-ESM2-0']:
    ds_calendar = 'proleptic_gregorian'
elif model in ['UKESM1-0-LL', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
    ds_calendar = '360_day'
    
print(ds_calendar, 'calendar')

piControl
noleap calendar


In [335]:
def area_weights(lat_bnds, lon_bnds): 
    # computes exact area weigths assuming earth is a perfect sphere
    lowerlats = np.radians(lat_bnds[:,0]); upperlats = np.radians(lat_bnds[:,1])
    difflon = np.radians(np.diff(lon_bnds[0,:])) # if the differences in longitudes are all the same
    areaweights = difflon*(np.sin(upperlats) - np.sin(lowerlats));
    areaweights /= areaweights.mean()
    return areaweights # list of weights, of same dimension as latitude

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def leap_year(year, calendar='standard'):
    """Determine if year is a leap year"""
    leap = False
    if ((calendar in ['standard', 'gregorian',
        'proleptic_gregorian', 'julian']) and
        (year % 4 == 0)):
        leap = True
        if ((calendar == 'proleptic_gregorian') and
            (year % 100 == 0) and
            (year % 400 != 0)):
            leap = False
        elif ((calendar in ['standard', 'gregorian']) and
                 (year % 100 == 0) and (year % 400 != 0) and
                 (year < 1583)):
            leap = False
    return leap

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
        if leap_year(year, calendar=calendar) and month == 2: # the feb-test is missing at the website!
            month_length[i] += 1
    return month_length

In [336]:
# inspiration taken from: http://xarray.pydata.org/en/stable/examples/monthly-means.html

# days per month:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
      }

def day_weights(ds, chosen_season = 'DJF', calendar = 'noleap'): # new function
    month_length = xr.DataArray(get_dpm((ds.time.to_index()), calendar=ds_calendar), coords=[ds.time], name='month_length')
    if chosen_season == 'DJF':
        season_months = month_length.where(month_length['time.season'] == season)
        # repeat last December month, and move it to the beginning
        season_months = xr.concat([season_months, season_months[-1]], dim = 'time')
        season_months = season_months.roll(time = 1)

        norm_by_annual = season_months.groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
        # repeat last December month to give it equal length as season_months
        norm_by_monthly = np.concatenate([norm_by_monthly, [norm_by_monthly[-1]]])

        weights = season_months/norm_by_monthly
        # make weigths have mean 1 in chosen season for all years
        # can be checked by running weights.rolling(min_periods=3, center=True, time=3).mean()
        # note that these weights start with a December month
    elif chosen_season == 'all':
        
        ##### This code is not tested yet #####
        norm_by_annual = season_months.groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by.values[i], 12) for i in range(len(norm_by.values))])
        weights = month_length/norm_by_monthly
        # normalized to have mean 1
    # if other season wanted, continue developing this if-test
    
    # NB: normalised weights do not care what numbers are produced for other seasons
    return weights 

In [378]:
latregion = slice(-5,5); lonregion = slice(190, 240) # = 120 W - 170 W
# use larger region before regridding, that adds 5 deg to each border:
larger_latregion = slice(-10,10); larger_lonregion = slice(185, 245)

resolution = 1;
ds_out = xr.Dataset({'lon': (['lon'], np.arange(lonregion.start+resolution/2, lonregion.stop+resolution/2, resolution)),
                     'lat': (['lat'], np.arange(latregion.start+resolution/2, latregion.stop+resolution/2, resolution))
                    }
                   )
    
regr_lat_bnds = np.array([[upper, upper+resolution] for upper in range(latregion.start,latregion.stop)])
regr_lon_bnds = np.array([[upper, upper+resolution] for upper in range(lonregion.start,lonregion.stop)])
area_w = area_weights(regr_lat_bnds, regr_lon_bnds)

season = 'DJF'
lastD = {}

for exp in exp_list:
#for exp in [exp_list[0]]:
    key = exp_keys[exp]
    exp_datasets = datasets[key]
    members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)
    #for member in [members_sorted.values[0]]: # check for first member only
    for member in members_sorted.values:
        print(exp, member)
        ds = exp_datasets.sel(member_id = member)
        
        # select regional data, perform a regridding, and compute area average
        if model == 'MCM-UA-1-0':
             ds = ds.rename({'longitude': 'lon','latitude': 'lat'}) 
        regional_data = ds.ts.sel(lat = larger_latregion, lon = larger_lonregion)
        regridder = xe.Regridder(regional_data, ds_out, 'bilinear', reuse_weights = True)
        regridded_data = regridder(regional_data)
        area_avg = (regridded_data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])
            
        yrs = int(area_avg.shape[0]/12)
        
        weights = day_weights(area_avg, calendar = ds_calendar)
        # double check that weights are 1 for all seasons, except the last
        meanweights = weights.rolling(min_periods=3, center=True, time=3).mean()
        print('years in experiment:', yrs, '    ',  'mean weights all 1?', all(meanweights.dropna(dim = 'time')[:-1] == 1))
        
        if exp == 'historical':
            # save last december month for each member for use in season mean in first year of ssp exps
            lastD[member] = area_avg[-1] 
            weights = weights[1:] # drop first december month
        elif exp == 'piControl':
            weights = weights[1:] # drop first december month
        elif exp not in ['piControl','historical']: # then it must be future scenario   
            area_avg = xr.concat([lastD[member], area_avg], dim = 'time')  
            weights = weights.assign_coords(time = area_avg.time)
            
        # average over season
        day_weighted_avg = area_avg*weights
        ds_season = day_weighted_avg.where(day_weighted_avg['time.season'] == season) # creates nan in all other months
            
        ds_season3 = ds_season.rolling(min_periods=3, center=True, time=3).mean()
        
        if exp not in ['piControl','historical']:
            # remove nan-value obtained from inserting last december month from historical
            ds_season3 = ds_season3[1:]
        seasonmean = ds_season3.groupby('time.year').mean('time') # make annual mean
        # no information the first year of piControl and historical, since we are missing the december month before
        
        # day-weighted rolling 3-months mean for all months (with seasonal variations)
        #day_weighted_avg_allyear = area_avg*day_weights(yrs, chosen_season = 'all')
        #smoothed_allyear = day_weighted_avg_allyear.rolling(min_periods=3, center=True, time=3).mean()
        
        colname = [(exp, member)]
        
        first_member_piControl = 'r1i1p1f1'
        if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'UKESM1-0-LL', 'MIROC-ES2L']:
            first_member_piControl = 'r1i1p1f2'
        elif model in ['GISS-E2-1-G']:
            first_member_piControl = 'r101i1p1f1'
        
        if exp == 'piControl' and member == first_member_piControl:
            # create dataframe for storing all results and make the piControl years the index
            df = pd.DataFrame(seasonmean.values, columns = colname)
        else:
            df_col = pd.DataFrame(seasonmean.values, columns = colname)
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
        
df.columns = pd.MultiIndex.from_tuples(df.columns, names=['Experiment','Member'])


piControl r1i1p1f1
Create weight file: bilinear_8x16_10x50.nc
years in experiment: 500      mean weights all 1? True
historical r1i1p1f1
Reuse existing file: bilinear_8x16_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r1i1p1f2
Reuse existing file: bilinear_8x16_10x50.nc
years in experiment: 165      mean weights all 1? True
ssp245 r1i1p1f2
Reuse existing file: bilinear_8x16_10x50.nc
years in experiment: 86      mean weights all 1? True
ssp370 r1i1p1f2
Reuse existing file: bilinear_8x16_10x50.nc
years in experiment: 86      mean weights all 1? True
ssp585 r1i1p1f2
Reuse existing file: bilinear_8x16_10x50.nc
years in experiment: 86      mean weights all 1? True


In [379]:
# check values in last December for historical
[lastD[member].values for member in lastD.keys()]

[array(301.64717399), array(303.65030327)]

In [380]:
pd.set_option('display.min_rows', 165)
df.iloc[:165]

Experiment   piControl  historical                  ssp245      ssp370  \
Member        r1i1p1f1    r1i1p1f1    r1i1p1f2    r1i1p1f2    r1i1p1f2   
0                  NaN         NaN         NaN  303.300248  303.343550   
1           299.875171  300.272375  300.404877  301.797072  301.322204   
2           300.136936  300.551805  300.600378  301.306079  301.105254   
3           300.228035  300.528635  300.510840  300.662331  300.717438   
4           301.300121  300.782540  301.081913  300.558373  300.573916   
5           300.961723  301.235655  301.422033  300.865869  300.791642   
6           301.515740  300.724076  300.404137  301.430942  302.156986   
7           300.194201  300.553894  300.401229  302.121756  301.749328   
8           299.385127  300.635436  301.556648  300.857614  301.555735   
9           299.732454  299.554730  300.781021  301.302249  302.072430   
10          300.107015  299.692499  300.650627  301.963972  301.631399   
11          300.743637  300.505577  300.355689  302.261348  301.919889   
12          300.701322  301.541754  301.424562  301.812712  302.075956   
13          301.535902  300.884842  301.082779  301.230873  301.839722   
14          301.386916  300.514888  300.517658  301.687249  301.851457   
15          301.208269  300.082810  300.061001  303.504567  302.600833   
16          300.301838  300.689163  300.762280  301.747714  302.102855   
17          300.382868  300.819914  300.538776  301.855209  301.793838   
18          300.961437  300.440422  300.264744  302.896677  302.381547   
19          302.071334  301.111777  300.137509  302.930219  302.088581   
20          301.102434  303.087017  300.875691  302.201167  301.254394   
21          301.304594  301.966112  301.494610  302.035443  301.525554   
22          300.546875  299.982354  301.108962  302.532326  301.440105   
23          299.897268  299.395457  300.381184  301.962615  301.845150   
24          299.565110  299.656120  299.670445  302.230528  302.628469   
25          299.832330  299.478372  300.220047  302.265662  302.635887   
26          299.425642  299.641482  300.583704  302.071607  302.564155   
27          300.557646  299.791490  300.546141  302.095311  301.509575   
28          301.234512  300.150950  301.084919  302.050608  301.845543   
29          301.078637  300.966206  300.985652  301.589110  301.866442   
..                 ...         ...         ...         ...         ...   
135         301.069756  300.203256  301.591897         NaN         NaN   
136         301.180169  302.171684  300.384005         NaN         NaN   
137         300.662328  301.614519  300.507897         NaN         NaN   
138         301.560159  300.947441  301.095515         NaN         NaN   
139         302.326909  301.357283  300.630606         NaN         NaN   
140         300.391027  300.313394  300.300175         NaN         NaN   
141         300.718712  300.508887  300.349913         NaN         NaN   
142         300.416063  300.880670  301.334364         NaN         NaN   
143         300.281836  302.129507  301.860094         NaN         NaN   
144         300.747114  301.189871  301.049704         NaN         NaN   
145         301.677466  301.405715  300.312639         NaN         NaN   
146         302.835365  300.643890  301.140388         NaN         NaN   
147         301.398548  301.183404  301.841460         NaN         NaN   
148         301.045351  301.602959  301.711713         NaN         NaN   
149         299.255352  302.363413  301.026124         NaN         NaN   
150         300.626980  301.828986  302.314099         NaN         NaN   
151         300.445641  300.390458  301.526898         NaN         NaN   
152         300.889410  300.447167  301.196733         NaN         NaN   
153         300.350101  300.728855  301.359749         NaN         NaN   
154         300.690895  301.048185  302.010296         NaN         NaN   
155         301.091562  301.269825  302.350282         NaN         NaN   
156 

## check that first and last rows of ssp exps look reasonable

In [381]:
#pd.set_option('display.min_rows', 90)
df.iloc[0]

Experiment  Member  
piControl   r1i1p1f1           NaN
historical  r1i1p1f1           NaN
            r1i1p1f2           NaN
ssp245      r1i1p1f2    303.300248
ssp370      r1i1p1f2    303.343550
ssp585      r1i1p1f2    303.332455
Name: 0, dtype: float64

In [382]:
pd.set_option('display.max_columns', 100)
#df['ssp370'].iloc[85:88]
df.iloc[85:88]

Experiment   piControl  historical                 ssp245      ssp370  \
Member        r1i1p1f1    r1i1p1f1    r1i1p1f2   r1i1p1f2    r1i1p1f2   
85          300.205252  299.861797  300.341464  301.41748  303.200502   
86          300.223531  300.464283  300.342781        NaN         NaN   
87          300.136643  300.681513  301.458152        NaN         NaN   

Experiment      ssp585  
Member        r1i1p1f2  
85          303.822968  
86                 NaN  
87                 NaN

## Save data to file

In [383]:
#df.to_csv('../Processed_data/Nino3_4_DJF/' + model + '_DJF_nino3_4index.txt')

In [302]:
# check for missing files

#cat = col.search(source_id = model, experiment_id = 'historical', variable_id='ts', table_id='Amon',member_id = 'r5i1p1f1')
#cat = col.search(source_id = model, experiment_id = 'historical', variable_id='ts', table_id='Amon')
#cat = col.search(source_id = model, experiment_id = 'piControl', variable_id='ts', table_id='Amon')

#cat = col.search(source_id = model, experiment_id = 'ssp126', variable_id='ts', table_id='Amon')
cat = col.search(source_id = model, experiment_id = 'ssp370', variable_id='ts', table_id='Amon')



In [303]:
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
42229,ScenarioMIP,NCAR,CESM2-WACCM,ssp370,r1i1p1f1,Amon,ts,gn,gs://cmip6/ScenarioMIP/NCAR/CESM2-WACCM/ssp370...,NaN
42234,ScenarioMIP,NCAR,CESM2-WACCM,ssp370,r2i1p1f1,Amon,ts,gn,gs://cmip6/ScenarioMIP/NCAR/CESM2-WACCM/ssp370...,NaN
42239,ScenarioMIP,NCAR,CESM2-WACCM,ssp370,r3i1p1f1,Amon,ts,gn,gs://cmip6/ScenarioMIP/NCAR/CESM2-WACCM/ssp370...,NaN
